In [2]:
import numpy as np,numpy  # 数据处理的库 numpy
import cv2   # 图像处理的库 OpenCv
import sys

In [3]:
# 从text文本中读取特征点
def readPoints(path) :
    # Create an array of points.
    points = [];
    # Read points
    with open(path) as file :
        for line in file :
            x, y = line.split()
            points.append((int(x), int(y)))
    return points

# Apply affine transform calculated using srcTri and dstTri to src and
# output an image of size.
def applyAffineTransform(src, srcTri, dstTri, size) :
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    # Apply the Affine Transform just found to the src image
    #像素使用线性插值的方式来对RGB颜色通道进行融合，flags=cv2.INTER_LINEAR
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )
    return dst

# Warps and alpha blends triangular regions from img1 and img2 to img
#t代表融合之后的特征三角，img1和img2代表图片， img代表morph图，t1和t2代表对应图片的特征三角
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :
    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))
    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []
    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))
    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);
    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]
    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t1Rect, tRect, size)
    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2
    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask


In [7]:
if __name__=='__main__':
    num=596
    #计算平均脸特征点
#     for p_num in range(1,num+1):
#         path='/home/fengchao/桌面/facedata/78points_shapefree/'+str(p_num)+'.txt'
#         if(p_num==1):
#             morph_points=readPoints(path)
#         else:
#             morph_points=morph_points+readPoints(path)
#         merge_point_path="/home/fengchao/桌面/facedata/merge_point.txt"
#         np.savetxt(merge_point_path,morph_points,fmt='%i %i')
    morph_points=readPoints('/home/fengchao/桌面/facedata/average_face.txt')
        
    #计算大众脸
    #为模板面孔赋值用于生成基于图片一的图片空间,融合图片的像素大小需要统一
    img=cv2.imread('/home/fengchao/桌面/facedata/78points_shapefree/1.jpg')
    morph_img= np.zeros(img.shape,dtype = img.dtype)
    # 从tri.txt文档中读入面部三角
    with open("/home/fengchao/桌面/facedata/humantri.txt") as file :
        for line in file :
            for img_num in range(1,num+1):
                filename1 = '/home/fengchao/桌面/facedata/78points_shapefree/'+str(img_num)+'.jpg'
                path1='/home/fengchao/桌面/facedata/average_face.txt'
                # 读取图片
                img1 = cv2.imread(filename1);#模板图
                # 将图片像素值转换成浮点数
                img1 = np.float64(img1)
                # 读取两张图片的特征点
                points1 = readPoints(path1)
                x,y,z = line.split()
                x = int(x)
                y = int(y)
                z = int(z)
                t1 = [points1[x], points1[y], points1[z]]
                t = [morph_points[x], morph_points[y], morph_points[z]]
                # Find bounding rectangle for each triangle
                r1 = cv2.boundingRect(np.float32([t1]))
                r = cv2.boundingRect(np.float32([t]))
                # Offset points by left top corner of the respective rectangles
                t1Rect = []
                tRect = []
                for i in range(0, 3):
                    tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
                    t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
                # Get mask by filling triangle
                mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
                cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);
                # Apply warpImage to small rectangular patches
                img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
                size = (r[2], r[3])
                warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
                # Alpha blend rectangular patches
                if(img_num==1):
                    imgRect=warpImage1
                else:
                    imgRect = imgRect+warpImage1
            imgRect=imgRect/num
            # Copy triangular region of the rectangular patch to the output image
            morph_img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = morph_img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

        # Display Result
        morph_path='/home/fengchao/桌面/facedata/morph_img.jpg'
        cv2.imwrite(morph_path, morph_img)
        cv2.namedWindow("Morphed Face",0);
        cv2.imshow("Morphed Face",np.uint8(morph_img))
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    #     save_path='/home/fengchao/桌面/merge results/shapedface/shapedface'+str(alpha)+'.jpg'
    #     #截取融合后的面部并重新设定大小
    #     shape_face(morph_path,merge_point_path,save_path)
